## Implementación de la clasificación multi-clase con Bayes ingenuo

In [1]:
import torch
import torchvision
from PIL import Image
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import numpy as np

### 1. Carga de las imágenes- mnist_dataset

In [3]:
def binarize_image(image_tensor):
    image_tensor[image_tensor > 0.5] = 1
    image_tensor[image_tensor <= 0.5] = 0
    return image_tensor

def imshow(image_tensor):
    im = plt.imshow(image_tensor.numpy(), cmap = 'gray')
    plt.show()


def load_dataset(path = "mnist_dataset/train"):
    #Open up the dataset
    dataset =  torchvision.datasets.ImageFolder(path)   
    #print(dataset)
    list_images = dataset.imgs
    #print(list_images)
    train_data_tensor  = None 
    train_data_tensor_gray = None
    labels_training = []
    first_tensor = True
    #list_images_training =  set(data_labeled.train_ds.x.items)
    #print(list_images)
    for i in range(len(list_images)):
        pair_path_label = list_images[i]
        image = Image.open(pair_path_label[0])        

        x_tensor = TF.to_tensor(image).squeeze() #Return tensor in range [0.0 - 1.0]
        x_tensor_gray = TF.pil_to_tensor(image).squeeze() #Return tensor in grayscale
        x_tensor_bin = binarize_image(x_tensor)
               
        #plt.figure()
        #plt.imshow(x_tensor_bin)
        x_tensor_bin_plain = x_tensor_bin.view(x_tensor_bin.shape[0] * x_tensor_bin.shape[1], -1)
        x_tensor_gray_plain = x_tensor_gray.view(x_tensor_gray.shape[0] * x_tensor_gray.shape[1], -1)
        #print("tensor ", x_tensor_bin_plain)
        #test dataset case        
        #if("train" in pair_path_label[0]):
        labels_training += [pair_path_label[1]]
        #print(pair_path_label)
        if(first_tensor):
            first_tensor = False
            train_data_tensor = x_tensor_bin_plain
            train_data_tensor_gray = x_tensor_gray_plain
        else:
            train_data_tensor = torch.cat((train_data_tensor, x_tensor_bin_plain), 1)  
            train_data_tensor_gray = torch.cat((train_data_tensor_gray, x_tensor_gray_plain), 1)
    return (train_data_tensor, train_data_tensor_gray, torch.tensor(labels_training))       

(train_data_tensor, train_data_tensor_gray, labels_training) = load_dataset()



### 2. Train_model

In [14]:

def evaluate_gaussian(mu, sigma, x):
    coef_norm = 1 / (torch.sqrt(torch.tensor(2.0 * torch.pi * sigma ** 2)))
    return coef_norm * torch.exp(torch.tensor(-0.5*((mu - x)/sigma )** 2))    

def train_model(train_data_tensor_bin, train_data_tensor_gray, labels_training, num_classes = 10):

    first_tensor = True
    p_t_tensor_acc = None
    p_m_1_given_k_acc = None
    p_m_0_given_k_acc = None
    gaussian_given_k_acc = None
    #print(train_data_tensor_gray[650, 0:60])
    
    for k in range(num_classes):
        # Filtra train_data_tensor por clase para dataset binarizado y en escala de grises
        train_data_tensor_bin_per_k = train_data_tensor_bin[:, labels_training == k].type(torch.int64)
        train_data_tensor_gray_per_k = train_data_tensor_gray[:, labels_training == k].type(torch.float64)
        #print(train_data_tensor_bin_per_k)
        #print(train_data_tensor_gray_per_k)
        
        # D = cantidad de filas (784 pixeles)
        # N = cantidad de columnas (600 imágenes)
        D, N = train_data_tensor_bin.shape
        
        # Estimacion de probabilidad a priori
        p_t_tensor = torch.tensor([train_data_tensor_bin_per_k.shape[1] / N])
        #print(p_t_tensor)
        
        # Estimacion de verisimilitud para cada pixel por clase
        p_m_1_given_k = (train_data_tensor_bin_per_k == 1).sum(dim=1)/train_data_tensor_bin_per_k.shape[1]
        #print(p_m_pix_val_given_k)
        
        # Transforma tensor de verisimilitud a una sola columna
        p_m_1_given_k = p_m_1_given_k.view(p_m_1_given_k.shape[0], -1)
        #print(p_m_pix_val_given_k)
    
        # Calculo de media y desviacion estandar
        mu_given_k = torch.mean(train_data_tensor_gray_per_k, dim=1)
        sigma_given_k = torch.std(train_data_tensor_gray_per_k, dim=1)+1e-12
        #print(mu_given_k.shape)
        #print(sigma_given_k.shape)
        
        # Transforma tensor de mu y sigma a una sola columna
        mu_given_k = mu_given_k.view(mu_given_k.shape[0], -1)
        sigma_given_k = sigma_given_k.view(sigma_given_k.shape[0], -1)
        #print(mu_given_k)
        
        # Estimacion de funcion de densidad de probabilidad
        gaussian_given_k = evaluate_gaussian(mu_given_k, sigma_given_k, train_data_tensor_gray_per_k)
        #print(p_gaussian)
    
        if(first_tensor):
            first_tensor = False
            p_m_1_given_k_acc = p_m_1_given_k
            p_t_tensor_acc = p_t_tensor
            gaussian_given_k_acc = gaussian_given_k
        else:
            p_m_1_given_k_acc = torch.cat((p_m_1_given_k_acc, p_m_1_given_k), 1)
            p_t_tensor_acc = torch.cat((p_t_tensor_acc, p_t_tensor), 0)
            gaussian_given_k_acc = torch.cat((gaussian_given_k_acc, gaussian_given_k), 1)

    # Saca complemento de p_m_1_given_k_acc
    p_m_0_given_k_acc = 1 - p_m_1_given_k_acc
    
    print("p_m_0_given_k_acc = ", p_m_0_given_k_acc.shape)
    print("p_m_1_given_k_acc = ", p_m_1_given_k_acc.shape)
    print("p_t_tensor_acc = ", p_t_tensor_acc.shape)
    print("gaussian_given_k_acc = ", gaussian_given_k_acc.shape)
        
    return (list([p_m_0_given_k_acc[650], p_m_1_given_k_acc[650]]), gaussian_given_k_acc[650], p_t_tensor_acc)

print("train gray dimensions", train_data_tensor_gray.shape)
print("train bin dimensions ", train_data_tensor.shape)
print("train labels ", len(labels_training))

train_model(train_data_tensor, train_data_tensor_gray, labels_training)

#train model by calculating the prior probabilities
#(p_m_pix_val_given_k, p_t_tensor) = train_model(train_data_tensor, labels_training)
#print("p_m_pix_val_given_k size ", p_m_pix_val_given_k.shape)
#(predicted_label, scores_classes) = test_model(train_data_tensor[:, 500], p_m_pix_val_given_k, p_t_tensor)
#print("predicted_label ", predicted_label)
#print("real label ", labels_training[500])
#acc = test_model_batch(train_data_tensor, labels_training, p_m_pix_val_given_k, p_t_tensor)
#print("Model accuracy ", acc)
     
    

#def test_model_batch(test_set, labels, p_m_pix_val_given_k, p_t_tensor):
    #TODO IMPLEMENT
    
    

train gray dimensions torch.Size([784, 600])
train bin dimensions  torch.Size([784, 600])
train labels  600
p_m_0_given_k_acc =  torch.Size([784, 10])
p_m_1_given_k_acc =  torch.Size([784, 10])
p_t_tensor_acc =  torch.Size([10])
gaussian_given_k_acc =  torch.Size([784, 600])


/var/folders/g3/jnbbw4bs1cqc87xtx7hgyr5w0000gn/T/ipykernel_24247/2020273718.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  coef_norm = 1 / (torch.sqrt(torch.tensor(2.0 * torch.pi * sigma ** 2)))
/var/folders/g3/jnbbw4bs1cqc87xtx7hgyr5w0000gn/T/ipykernel_24247/2020273718.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return coef_norm * torch.exp(torch.tensor(-0.5*((mu - x)/sigma )** 2))


([tensor([0.9500, 1.0000, 0.9333, 0.8000, 1.0000, 0.9500, 1.0000, 0.9667, 0.9000,
          1.0000]),
  tensor([0.0500, 0.0000, 0.0667, 0.2000, 0.0000, 0.0500, 0.0000, 0.0333, 0.1000,
          0.0000])],
 tensor([8.9287e-03, 8.9287e-03, 8.9287e-03, 9.3870e-03, 2.4916e-04, 1.0749e-06,
         8.9287e-03, 9.5478e-03, 1.9093e-03, 8.9287e-03, 3.2182e-03, 6.3353e-03,
         8.9287e-03, 8.9287e-03, 8.9287e-03, 8.9287e-03, 9.2537e-03, 9.6279e-03,
         8.9287e-03, 8.9287e-03, 9.2537e-03, 9.6279e-03, 9.5478e-03, 8.9287e-03,
         9.6279e-03, 9.7292e-03, 9.4459e-03, 9.3229e-03, 8.9287e-03, 9.1795e-03,
         8.9287e-03, 8.9287e-03, 9.3229e-03, 8.9287e-03, 9.1006e-03, 9.3870e-03,
         9.6596e-03, 8.9287e-03, 9.5478e-03, 8.9287e-03, 8.9287e-03, 1.0074e-05,
         9.4478e-03, 8.9287e-03, 3.4409e-04, 9.0169e-03, 8.9287e-03, 8.9287e-03,
         9.6279e-03, 9.4459e-03, 9.4996e-03, 9.0169e-03, 8.9287e-03, 8.9287e-03,
         8.9287e-03, 8.9287e-03, 8.9287e-03, 8.9287e-03, 8.9287e-0

### 3. Test Gaussian Model Function

In [11]:
def test_model(input_torch,p_t_tensor_acc,gaussian_given_k_acc, num_classes = 10):
    Probabilities: 
    return

### 4. Test Bernoulli Model Function

### 4.Test Model batch Function: Accuracy